In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import os

# Load data with explicit path
file_path = r"C:\Users\Admin\Downloads\QVI_data.csv"  # Raw string for Windows path

# Verify file exists
if not os.path.exists(file_path):
    print(f"Error: File not found at {file_path}")
else:
    # Load the dataset
    qvi_data = pd.read_csv(file_path)
    
    # Convert DATE column with validation
    qvi_data['DATE'] = pd.to_datetime(qvi_data['DATE'], 
                                       format='%Y-%m-%d', 
                                       errors='coerce')
    
    if qvi_data['DATE'].isnull().any():
        print(f"Found {qvi_data['DATE'].isnull().sum()} invalid dates")
        print("Sample invalid rows:")
        print(qvi_data[qvi_data['DATE'].isnull()].head(2))
        qvi_data = qvi_data.dropna(subset=['DATE'])
    
    # 1. Data Preparation
    # Aggregate monthly metrics for all stores
    monthly_data = qvi_data.groupby(['STORE_NBR', pd.Grouper(key='DATE', freq='ME')]).agg(
        total_sales=('TOT_SALES', 'sum'),
        total_customers=('LYLTY_CARD_NBR', 'nunique')
    ).reset_index()
    
    monthly_data['avg_transactions'] = monthly_data['total_sales'] / monthly_data['total_customers']
    
    # 2. Control Store Selection Function
    def find_control_store(trial_store, pre_trial_period=('2018-07-01', '2019-01-31')):
        pre_trial_mask = (monthly_data['DATE'] >= pre_trial_period[0]) & (monthly_data['DATE'] <= pre_trial_period[1])
        pre_trial = monthly_data[pre_trial_mask]
        
        trial_metrics = pre_trial[pre_trial['STORE_NBR'] == trial_store][['total_sales', 'total_customers', 'avg_transactions']].mean()
        
        scores = []
        for store in pre_trial['STORE_NBR'].unique():
            if store == trial_store:
                continue
            control_metrics = pre_trial[pre_trial['STORE_NBR'] == store][['total_sales', 'total_customers', 'avg_transactions']].mean()
            
            distances = 1 - (abs(trial_metrics - control_metrics) - 
                             (trial_metrics.min() - control_metrics.min())) / (
                             (trial_metrics.max() - control_metrics.min()) - 
                             (trial_metrics.min() - control_metrics.min()))
            
            scores.append({
                'store': store,
                'score': np.mean(distances),
                'sales_corr': trial_metrics['total_sales'] / control_metrics['total_sales']
            })
        
        scores_df = pd.DataFrame(scores)
        return scores_df.sort_values('score', ascending=False).iloc[0]['store']

    # 3. Find control stores
    trial_stores = [77, 86, 88]
    control_mapping = {store: find_control_store(store) for store in trial_stores}

    # 4. Trial Analysis
    def analyze_trial(trial_store, control_store, trial_period=('2019-02-01', '2019-04-30')):
        trial_data = monthly_data[
            (monthly_data['STORE_NBR'] == trial_store) & 
            (monthly_data['DATE'].between(*trial_period))
        ]
        
        control_data = monthly_data[
            (monthly_data['STORE_NBR'] == control_store) & 
            (monthly_data['DATE'].between(*trial_period))
        ]
        
        t_stat, p_value = stats.ttest_ind(trial_data['total_sales'], control_data['total_sales'])
        
        # Corrected driver analysis section
        driver_analysis = pd.DataFrame({
            'metric': ['total_sales', 'total_customers', 'avg_transactions'],
            'trial': [
                trial_data['total_sales'].mean(),
                trial_data['total_customers'].mean(),
                trial_data['avg_transactions'].mean()
            ],
            'control': [
                control_data['total_sales'].mean(),
                control_data['total_customers'].mean(),
                control_data['avg_transactions'].mean()
            ]
        })
        
        return t_stat, p_value, driver_analysis

    # 5. Generate Results
    results = []
    for trial_store in trial_stores:
        control_store = control_mapping[trial_store]
        t_stat, p_value, driver = analyze_trial(trial_store, control_store)
        
        results.append({
            'trial_store': trial_store,
            'control_store': control_store,
            't_stat': t_stat,
            'p_value': p_value,
            'driver_analysis': driver
        })

    # 6. Display Results
    for result in results:
        print(f"Trial Store: {result['trial_store']}, Control Store: {result['control_store']}")
        print(f"T-statistic: {result['t_stat']}, P-value: {result['p_value']}")
        print("Driver Analysis:")
        print(result['driver_analysis'])
        print("\n" + "-"*50 + "\n")

Trial Store: 77, Control Store: 233.0
T-statistic: 2.1043553466105163, P-value: 0.10314505225726589
Driver Analysis:
             metric       trial     control
0       total_sales  259.000000  200.566667
1   total_customers   47.333333   38.333333
2  avg_transactions    5.466202    5.228796

--------------------------------------------------

Trial Store: 86, Control Store: 196.0
T-statistic: 0.6227973325430964, P-value: 0.5671657089321503
Driver Analysis:
             metric      trial     control
0       total_sales  929.40000  873.133333
1   total_customers  109.00000   92.666667
2  avg_transactions    8.51379    9.398627

--------------------------------------------------

Trial Store: 88, Control Store: 237.0
T-statistic: 2.1370250720525052, P-value: 0.09941957736359801
Driver Analysis:
             metric        trial      control
0       total_sales  1428.933333  1272.533333
1   total_customers   128.666667   121.666667
2  avg_transactions    11.106398    10.446827

-----------